In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
import rtree
import pickle

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import secret_key

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
mta_df = pd.read_csv("data/mta_turnstile/turnstile_170107.txt")
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170114.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170121.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170128.txt"))

In [3]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543


In [4]:
mta_df['DATE'] = pd.to_datetime(mta_df.DATE)

In [5]:
mta_df['month'] = mta_df.DATE.dt.month

In [6]:
mta_df['year'] = mta_df.DATE.dt.year

In [7]:
mta_df['day_of_week'] = mta_df.DATE.dt.weekday_name

In [8]:
mta_df.day_of_week.value_counts()

Tuesday      111687
Wednesday    111432
Friday       110914
Thursday     110841
Monday       110819
Sunday       110728
Saturday     110682
Name: day_of_week, dtype: int64

In [12]:
mta_df[mta_df.STATION == 'CENTRAL AV'].head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week
38893,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,00:00:00,REGULAR,507074,1691710,12,2016,Saturday
38894,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,04:00:00,REGULAR,507081,1691803,12,2016,Saturday
38895,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,08:00:00,REGULAR,507099,1691827,12,2016,Saturday
38896,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,12:00:00,REGULAR,507175,1691884,12,2016,Saturday
38897,K017,R401,00-00-00,CENTRAL AV,M,BMT,2016-12-31,16:00:00,REGULAR,507256,1692008,12,2016,Saturday


In [23]:
import requests
import json
response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=103+st+station+new+york+line+1&key='+secret_key.google_api_key)

In [24]:
#response.json()[['results']
test = json.loads(response.text)

In [27]:
test['results'][0]['geometry']['location']['lat'], test['results'][0]['geometry']['location']['lng']

(40.7902308, -73.9477564)

In [28]:
mta_loc = mta_df.groupby(['STATION', 'LINENAME', 'DIVISION'])['UNIT'].count().reset_index()

In [31]:
mta_loc = mta_loc.drop('UNIT', axis=1)

In [35]:
mta_loc.head()

,STATION,LINENAME,DIVISION
0,1 AV,L,BMT
1,103 ST,1,IRT
2,103 ST,6,IRT
3,103 ST,BC,IND
4,103 ST-CORONA,7,IRT


In [53]:
import time

In [69]:
for index, loc in mta_loc.iterrows():
    try:
        station = mta_loc.STATION[index]
        linename = mta_loc.LINENAME[index]
        division = mta_loc.DIVISION[index]

        search_query = "https://maps.googleapis.com/maps/api/geocode/json?address="
        for word in station.split(" "):
            search_query += word + '+'

        search_query += 'station+' + linename + '+new+york+'
        search_query += division + '&key=' + secret_key.google_api_key
#         print(search_query)
        response = requests.get(search_query)
        result = json.loads(response.text)

        mta_loc.loc[[index], 'lat'] = result['results'][0]['geometry']['location']['lat']
        mta_loc.loc[[index], 'long'] = result['results'][0]['geometry']['location']['lng']
    except:
        print(mta_loc.STATION[index])
    time.sleep(1)
        
    

111 ST
CENTRAL AV
HEWES ST


In [96]:
mta_loc[mta_loc.STATION == 'HEWES ST']

,STATION,LINENAME,DIVISION,lat,long
328,HEWES ST,JM,BMT,NaN,-73.952892


In [97]:
mta_loc['lat'][328] = 40.706693

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [95]:
mta_loc['long'][328] = -73.952892

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
mta_loc['long'][244]

-73.927481